In [4]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np

# Load Foil Data

In [6]:
from ICARUS.Software.XFLR5.polars import read_polars_2d
from ICARUS.Database.db import DB
from ICARUS.Database import XFLRDB

In [7]:
db = DB()
foildb = db.foilsDB
foildb.load_data()
read_polars_2d(foildb,XFLRDB)
airfoils = foildb.set_available_airfoils()

# Get Plane

In [8]:
from Data.Planes.hermes import hermes
airplane = hermes(airfoils ,"Hermes")

# Import Enviroment

In [9]:
from ICARUS.Enviroment.definition import EARTH
print(EARTH)

Environment: Earth


# Get Solver

In [10]:
from ICARUS.Software.GenuVP3.gnvp3 import get_gnvp3
gnvp3 = get_gnvp3(db)
print(gnvp3)

3D Solver gnvp3:
Available Analyses Are: 
------------------- 
0) rerun 
1) Angles_Serial 
2) Angles_Parallel 
3) Pertrubation_Serial 
4) Pertrubation_Parallel 



## AoA Run

In [11]:
analysis = gnvp3.available_analyses_names()[2] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.set_analyses(analysis)
options = gnvp3.get_analysis_options(verbose=True)
solver_parameters = gnvp3.get_solver_parameters()

Selecting Analysis: Angles_Parallel



In [12]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)
u_freestream = 20
# airplane.defineSim(u_freestream, EARTH.air_density)

options.plane.value         = airplane
options.environment.value   = EARTH
options.db.value            = db
options.solver2D.value      = 'XFLR'
options.maxiter.value       = 100
options.timestep.value      = 5e-2
options.u_freestream.value  = u_freestream
options.angles.value        = angles

gnvp3.print_analysis_options()
# gnvp3.run()
polars = gnvp3.get_results()
airplane.save()


Getting Results
Making Polars


# Dynamics

### Define and Trim Plane

In [13]:
from ICARUS.Flight_Dynamics.state import State
if isinstance(polars,int):
    raise Exception("No Polars")
unstick = State("Unstick",airplane,polars,EARTH)

Index(['CL', 'CD', 'Cm', 'Cn', 'Cl', 'AoA'], dtype='object')
Cm is 0.005523873782960887 instead of 0 at AoA = -1.0
Interpolated values are: AoA = -0.7075245433130413 , Cm = -8.673617379884035e-19, Cl = 0.47095391336948783
Trim velocity is 20.322478445096582 m/s


### Pertrubations

In [14]:
unstick.add_all_pertrubations("Central")
unstick.get_pertrub()

u disturbance:	Type:	Derivative and 	Amplitude:	0.01.
u disturbance:	Type:	Derivative and 	Amplitude:	-0.01.
w disturbance:	Type:	Derivative and 	Amplitude:	0.01.
w disturbance:	Type:	Derivative and 	Amplitude:	-0.01.
q disturbance:	Type:	Derivative and 	Amplitude:	0.25.
q disturbance:	Type:	Derivative and 	Amplitude:	-0.25.
theta disturbance:	Type:	Value and 	Amplitude:	0.01.
theta disturbance:	Type:	Value and 	Amplitude:	-0.01.
v disturbance:	Type:	Derivative and 	Amplitude:	0.01.
v disturbance:	Type:	Derivative and 	Amplitude:	-0.01.
p disturbance:	Type:	Derivative and 	Amplitude:	0.25.
p disturbance:	Type:	Derivative and 	Amplitude:	-0.25.
r disturbance:	Type:	Derivative and 	Amplitude:	0.25.
r disturbance:	Type:	Derivative and 	Amplitude:	-0.25.
phi disturbance:	Type:	Value and 	Amplitude:	0.01.
phi disturbance:	Type:	Value and 	Amplitude:	-0.01.
Trim:	Type:	None and 	Amplitude:	None.


In [15]:
analysis = gnvp3.available_analyses_names(verbose=True)[4] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.set_analyses(analysis)
options = gnvp3.get_analysis_options(verbose=True)

3D Solver gnvp3:
Available Analyses Are: 
------------------- 
0) rerun 
1) Angles_Serial 
2) Angles_Parallel 
3) Pertrubation_Serial 
4) Pertrubation_Parallel 

Selecting Analysis: Pertrubation_Parallel



In [17]:
options.plane.value         = airplane
options.state.value         = unstick
options.environment.value   = EARTH
options.db.value            = db
options.solver2D.value      = 'XFLR'
options.maxiter.value       = 200
options.timestep.value      = 5e-3
options.u_freestream.value  = unstick.trim['U']
options.angle.value        = unstick.trim['AoA']

gnvp3.print_analysis_options()
gnvp3.run()
gnvp3.get_results()
unstick.save()


Running Solver gnvp3:
	Analysis Pertrubation_Parallel...


TypeError: run_pertrubation_parallel() got an unexpected keyword argument 'angles'

# Sensitivity Analysis

In [ ]:
# if sensAnalysisGNVP3 :
#     for var in ['u','w','q','theta','v','p','r','phi']:
#         space = np.logspace(np.log10(0.00001), np.log10(1), 10, base=10)
#         space =[*-space,*space]
#         maxiter = 2
#         timestep = 5e-2
#         dyn.sensitivityAnalysis(var, space)
#         GNVP3BatchArgs = [dyn, var, foildb, "Xfoil",
#                         maxiter, timestep,
#                         dyn.trim['U'], dyn.trim['AoA']]
#         dyn.runAnalysis(runGNVPsensitivityParallel, GNVP3BatchArgs)
#         perDIR = os.path.join(dyn.CASEDIR, f"Sensitivity_{var}")
#         dyn.sensResults[var] = pertrResults(perDIR, HOMEDIR)
#     dyn.save()